In [1]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
# !pip install tensorflow-gpu==2.0.0-rc1
!pip install 'h5py==2.10.0' --force-reinstall

'pip'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [3]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [4]:
import mysql.connector
import numpy as np

# 데이터베이스 연결 설정
db_connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='0252',
    database='lotto'
)

# 커서 생성
cursor = db_connection.cursor()

# SQL 쿼리 실행 (lotto 테이블에서 모든 데이터 가져오기)
cursor.execute("SELECT * FROM lotto")
results = cursor.fetchall()

# numpy 배열로 변환
rows = np.array(results)

# 행 개수 출력
row_count = len(rows)
print(row_count)



537


In [5]:
import numpy as np


def numbers2ohbin(numbers):

    ohbin = np.zeros(45) #45개의 빈 칸을 만듬

    for i in range(6): #여섯개의 당첨번호에 대해서 반복함
        ohbin[int(numbers[i])-1] = 1 #로또번호가 1부터 시작하지만 벡터의 인덱스 시작은 0부터 시작하므로 1을 뺌
    
    return ohbin

#원핫인코딩으로 변환
def ohbin2numbers(ohbin):

    numbers = []
    
    for i in range(len(ohbin)):
        if ohbin[i] == 1.0: # 1.0으로 설정되어 있으면 해당 번호를 반환값에 추가한다.
            numbers.append(i+1)
    
    return numbers

In [6]:
numbers = rows[:, 1:7]
ohbins = list(map(numbers2ohbin, numbers))

x_samples = ohbins[0:row_count-1]
y_samples = ohbins[1:row_count]

#원핫인코딩으로 표시
print("ohbins")
print("X[0]: " + str(x_samples[0]))
print("Y[0]: " + str(y_samples[0]))

#번호로 표시
print("numbers")
print("X[0]: " + str(ohbin2numbers(x_samples[0])))
print("Y[0]: " + str(ohbin2numbers(y_samples[0])))

ohbins
X[0]: [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Y[0]: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
numbers
X[0]: [2, 16, 19, 31, 34, 35]
Y[0]: [13, 14, 22, 27, 30, 38]


In [7]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [8]:
import tensorflow as tf

# TensorFlow 버전 출력
print(tf.__version__)


2.13.0


In [9]:
train_idx = (0, 400) #훈련
val_idx = (401, 500) #검증
test_idx = (501, len(x_samples)) #시험

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

# 모델을 정의합니다.
model = keras.Sequential([
    keras.layers.LSTM(128, batch_input_shape=(1, 1, 45), return_sequences=False, stateful=True),
    keras.layers.Dense(45, activation='sigmoid')
])

# 모델을 컴파일합니다.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
# 매 에포크마다 훈련과 검증의 손실 및 정확도를 기록하기 위한 변수
train_loss = []
train_acc = []
val_loss = []
val_acc = []

# 최대 100번 에포크까지 수행
#for epoch in range(100):
for epoch in range(1):

    model.reset_states()

    batch_train_loss = []
    batch_train_acc = []
    
    for i in range(train_idx[0], train_idx[1]):
        
        xs = x_samples[i].reshape(1, 1, 45)
        ys = y_samples[i].reshape(1, 45)
        
        loss, acc = model.train_on_batch(xs, ys) #배치만큼 모델에 학습시킴

        batch_train_loss.append(loss)
        batch_train_acc.append(acc)

    train_loss.append(np.mean(batch_train_loss))
    train_acc.append(np.mean(batch_train_acc))

    batch_val_loss = []
    batch_val_acc = []

    for i in range(val_idx[0], val_idx[1]):

        xs = x_samples[i].reshape(1, 1, 45)
        ys = y_samples[i].reshape(1, 45)
        
        loss, acc = model.test_on_batch(xs, ys) #배치만큼 모델에 입력하여 나온 답을 정답과 비교함
        
        batch_val_loss.append(loss)
        batch_val_acc.append(acc)

    val_loss.append(np.mean(batch_val_loss))
    val_acc.append(np.mean(batch_val_acc))

    print('epoch {0:4d} train acc {1:0.3f} loss {2:0.3f} val acc {3:0.3f} loss {4:0.3f}'.format(epoch, np.mean(batch_train_acc), np.mean(batch_train_loss), np.mean(batch_val_acc), np.mean(batch_val_loss)))

epoch    0 train acc 0.037 loss 0.418 val acc 0.040 loss 0.399


In [12]:
def gen_numbers_from_probability(nums_prob):

    ball_box = []

    for n in range(45):
        ball_count = int(nums_prob[n] * 100 + 1)
        ball = np.full((ball_count), n+1) #1부터 시작
        ball_box += list(ball)

    selected_balls = []

    while True:
        
        if len(selected_balls) == 6:
            break
        
        ball_index = np.random.randint(len(ball_box), size=1)[0]
        ball = ball_box[ball_index]

        if ball not in selected_balls:
            selected_balls.append(ball)

    return selected_balls

In [13]:
#검증셋 없이 모든 데이터를 다시 훈련시킨다.
# 최대 100번 에포크까지 수행
#for epoch in range(100):
for epoch in range(1):

    model.reset_states() # 중요! 매 에포크마다 1회부터 다시 훈련하므로 상태 초기화 필요

    for i in range(len(x_samples)):
        
        xs = x_samples[i].reshape(1, 1, 45)
        ys = y_samples[i].reshape(1, 45)
        
        loss, acc = model.train_on_batch(xs, ys) #배치만큼 모델에 학습시킴

        batch_train_loss.append(loss)
        batch_train_acc.append(acc)

    train_loss.append(np.mean(batch_train_loss))
    train_acc.append(np.mean(batch_train_acc))

    print('epoch {0:4d} train acc {1:0.3f} loss {2:0.3f}'.format(epoch, np.mean(batch_train_acc), np.mean(batch_train_loss)))


epoch    0 train acc 0.032 loss 0.406


In [14]:
# 마지막 회차까지 학습한 모델로 다음 회차 추론

print('receive numbers')

xs = x_samples[-1].reshape(1, 1, 45)

ys_pred = model.predict_on_batch(xs)

list_numbers = []

for n in range(10):
    numbers = gen_numbers_from_probability(ys_pred[0])
    numbers.sort()
    print('{0} : {1}'.format(n, numbers))
    list_numbers.append(numbers)

receive numbers
0 : [9, 13, 18, 24, 30, 39]
1 : [8, 21, 22, 28, 44, 45]
2 : [4, 7, 8, 20, 22, 37]
3 : [3, 6, 12, 25, 28, 35]
4 : [4, 13, 19, 21, 22, 41]
5 : [7, 18, 19, 24, 32, 39]
6 : [16, 23, 30, 31, 33, 35]
7 : [10, 13, 15, 24, 25, 40]
8 : [11, 20, 23, 31, 34, 40]
9 : [10, 16, 24, 27, 34, 44]


In [15]:
# 결과값을 하나의 변수에 저장
all_results = []

print('receive numbers')

xs = x_samples[-1].reshape(1, 1, 45)

ys_pred = model.predict_on_batch(xs)

for n in range(10):
    numbers = gen_numbers_from_probability(ys_pred[0])
    numbers.sort()
    print('{0} : {1}'.format(n, numbers))
    all_results.append(numbers)

# 결과값을 출력
print("모든 결과:", all_results)


receive numbers
0 : [10, 16, 20, 22, 44, 45]
1 : [8, 24, 29, 33, 35, 37]
2 : [13, 14, 19, 26, 36, 44]
3 : [4, 6, 16, 30, 38, 41]
4 : [4, 9, 16, 20, 40, 45]
5 : [12, 15, 33, 37, 40, 44]
6 : [1, 4, 9, 28, 39, 44]
7 : [14, 17, 32, 33, 34, 45]
8 : [6, 19, 30, 32, 38, 44]
9 : [10, 15, 17, 28, 43, 44]
모든 결과: [[10, 16, 20, 22, 44, 45], [8, 24, 29, 33, 35, 37], [13, 14, 19, 26, 36, 44], [4, 6, 16, 30, 38, 41], [4, 9, 16, 20, 40, 45], [12, 15, 33, 37, 40, 44], [1, 4, 9, 28, 39, 44], [14, 17, 32, 33, 34, 45], [6, 19, 30, 32, 38, 44], [10, 15, 17, 28, 43, 44]]


In [16]:
!pip install flask
!pip install flask-cors

'pip'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'pip'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [17]:
pip install flask flask-cors


Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install flask-cors

Note: you may need to restart the kernel to use updated packages.


In [19]:



all_results1 = str(all_results)

print( all_results1)

[[10, 16, 20, 22, 44, 45], [8, 24, 29, 33, 35, 37], [13, 14, 19, 26, 36, 44], [4, 6, 16, 30, 38, 41], [4, 9, 16, 20, 40, 45], [12, 15, 33, 37, 40, 44], [1, 4, 9, 28, 39, 44], [14, 17, 32, 33, 34, 45], [6, 19, 30, 32, 38, 44], [10, 15, 17, 28, 43, 44]]


In [20]:
from flask import Flask, jsonify
from flask_cors import CORS
import threading

app = Flask(__name__)
CORS(app)  # CORS 설정

@app.route('/get-variable')
def get_variable():
    return jsonify({'all_results1': all_results1})

def run_app():
    app.run(port=5001, debug=False) 

# Flask 앱을 백그라운드에서 실행
threading.Thread(target=run_app).start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [07/Oct/2024 21:57:39] "GET /get-variable HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2024 21:57:43] "GET /get-variable HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2024 21:57:45] "GET /get-variable HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2024 21:57:45] "GET /get-variable HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2024 21:57:46] "GET /get-variable HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2024 21:57:48] "GET /get-variable HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2024 21:57:52] "GET /get-variable HTTP/1.1" 200 -
